In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
base_model = "/data/terencewang/qwen"
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token_id = 151644
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
model = model.to("cuda")
model.config.pad_token_id = 151644

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
model.eval()
f = open("./lds_test/data/item2id.txt", "r")
# the format of the item name file is
# item_name item_id
# A 0
# B 1
lines = f.readlines()
f.close()
text = [
    _.split("\t")[0].strip(" ").strip('"') for _ in lines
]  # remove the leading and trailing spaces and quotess make sure this preprocess is the same as the prediction

In [8]:
from tqdm import tqdm


def batch(list, batch_size=1):
    chunk_size = (len(list) - 1) // batch_size + 1
    for i in range(chunk_size):
        yield list[batch_size * i : batch_size * (i + 1)]


item_embedding = []
for i, batch_input in tqdm(enumerate(batch(text, 1))):
    input = tokenizer(batch_input, return_tensors="pt", padding=True)
    input_ids = input.input_ids.to("cuda")
    attention_mask = input.attention_mask.to("cuda")
    outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
    hidden_states = outputs.hidden_states
    item_embedding.append(hidden_states[-1][:, -1, :].detach())
    # break
item_embedding = torch.cat(item_embedding, dim=0)
torch.save(item_embedding, "./lds_test/data/item_embedding_pad.pt")

14239it [09:01, 26.31it/s]


In [9]:
item_embedding.shape

torch.Size([14239, 1024])